In [2]:
import pandas as pd

df = pd.read_csv("Messy_HR_Dataset_Detailed.csv")

# Descriptive Statistic : Total Budget Training Loss and Benefit By Training Outcome
failed_training_df = df[df["Training Outcome"] == "Failed"]
total_loss_cost = failed_training_df['Training Cost'].sum().round(2)
total_person_failed = failed_training_df['Training Cost'].count()
average_failed = (total_loss_cost / total_person_failed).round(2)

success_training_df = df[df["Training Outcome"].isin(["Passed", "Completed"])]
total_benefit_cost = success_training_df['Training Cost'].sum().round(2)
total_person_success = success_training_df['Training Cost'].count()
average_success = (total_benefit_cost / total_person_success).round(2)

print(f"Total Company Money's Loss By Failed Training is ${total_loss_cost} from {total_person_failed} person") 
print(f"The average cost per-Failed person is ${average_failed}")
print(f"Total Budget Spending on Success Training is ${total_benefit_cost} from {total_person_success} person") 
print(f"The average cost per-Succeed person is ${average_success}")

Total Company Money's Loss By Failed Training is $421729.55 from 751 person
The average cost per-Failed person is $561.56
Total Budget Spending on Success Training is $893810.5 from 1587 person
The average cost per-Succeed person is $563.21


In [106]:
import pandas as pd

df = pd.read_csv("Messy_HR_Dataset_Detailed.csv")

# Descriptive Statistic : Top 3 Best Employee and Employee With Lowest Score
# Define scoring for each Training Outcome
score_mapping = {
    "Failed": -1,
    "Incomplete": 0,
    "Completed": 1,
    "Passed": 2
}
perf_mapping = {
    "PIP": -1,
    "Needs Improvement": 0,
    "Exceeds": 1,
    "Fully Meets": 2
}
df["Score"] = df["Training Outcome"].map(score_mapping) + df["Satisfaction Score"] + df["Engagement Score"] + df["Current Employee Rating"] + df["Performance Score"].map(perf_mapping)

# Calculate the total score for each employee
df["FullName"] = df["FirstName"] + " " + df["LastName"]
total_scores = df.groupby("FullName")["Score"].sum().sort_values(ascending=False)

top_3 = total_scores.head(3)
low_3 = total_scores.tail(3)
print("Top 3 Best Employee")
print(top_3)
print("\nEmployee With Lowest Score")
print(low_3)

Top 3 Best Employee
FullName
Isaias Pineda     36
Kane Black        34
Kaylyn Patrick    34
Name: Score, dtype: int64

Employee With Lowest Score
FullName
Chace Church       4
Samara Roberson    3
Tanya Leonard      3
Name: Score, dtype: int64


In [105]:
import pandas as pd
from datetime import datetime

df = pd.read_csv("Messy_HR_Dataset_Detailed.csv")

# Descriptive Statistic : Show The 7 Employee Who Have Served The Longest
df["FullName"] = df["FirstName"] + " " + df["LastName"]
df['StartDate'] = pd.to_datetime(df['StartDate'], format='%d-%b-%y')

# Convert ExitDate to datetime
df['CurrDate'] = pd.to_datetime(df['ExitDate'], format='%d-%b-%y', errors='coerce')

# Replace Empty Exit Date as today date
df['CurrDate'] = df['CurrDate'].fillna(pd.Timestamp.today())

# Calculate days worked
df['DaysWorked'] = (df['CurrDate'] - df['StartDate']).dt.days
df = df.sort_values(by='DaysWorked', ascending=False)
df[['FullName','Title','StartDate','DaysWorked']].head(7)

,FullName,Title,StartDate,DaysWorked
2863,Jaiden Middleton,Production Technician I,2018-08-07,2412
1465,Saniya Buck,Production Technician II,2018-08-07,2412
2484,Estrella Ho,Production Technician I,2018-08-07,2412
1741,Rashad Mayo,Production Technician II,2018-08-08,2411
124,Emery Roach,Area Sales Manager,2018-08-11,2408
533,Andreas Torres,CIO,2018-08-11,2408
1414,Remington Bullock,Production Technician I,2018-08-14,2405


In [153]:
import pandas as pd
from datetime import datetime

df = pd.read_csv("Messy_HR_Dataset_Detailed.csv")

df["FullName"] = df["FirstName"] + " " + df["LastName"]

# Assign Base Salary
salary_mapping = {
    "Production Technician I": 8500000, 
    "Production Technician II": 9500000, 
    "Area Sales Manager": 18000000,
    "CIO": 70000000,
    "Production Manager": 25000000,
    "Network Engineer": 15000000,
    "Sr. Accountant": 20000000,
    "Software Engineer": 18000000,
    "Senior BI Developer": 22000000,
    "Data Analyst": 16000000,
    "Director of Sales": 50000000,
    "Enterprise Architect": 30000000,
    "Administrative Assistant": 7000000,
    "IT Support": 10000000,
    "Sales Manager": 20000000,
    "BI Director": 40000000,
    "President & CEO": 100000000,
    "Sr. Network Engineer": 18000000,
    "BI Developer": 17000000,
    "IT Manager - Support": 25000000,
    "Shared Services Manager": 23000000,
    "IT Manager - DB": 26000000,
    "Accountant I": 12000000,
    "Principal Data Architect": 32000000,
    "Database Administrator": 17000000,
    "Sr. DBA": 22000000,
    "IT Manager - Infra": 28000000,
    "Software Engineering Manager": 30000000,
    "Director of Operations": 55000000,
    "IT Director": 60000000,
    "Data Architect": 25000000
}
df["Salary Base"] = df["Title"].str.strip().map(salary_mapping)

# Assign Base Salary Bonus Based on Employee Type
def assign_base_salary_employee_type(salary, etype):
    if etype == 'Contract':
        return 0
    elif etype == 'Full-Time':
        return (salary * 10.1 / 100) + (salary * 5 / 100)  
    elif etype == 'Part-Time':
        base = salary * 7.5 / 100
        return -1 * (base - (base * 25 / 100))
df['Subsidy (Employee Type)'] = df.apply(lambda row: assign_base_salary_employee_type(row['Salary Base'],row['EmployeeType']), axis=1)

# Assign Base Salary Bonus Based on Marital Status
def assign_base_salary_marital(salary,status):
    if status == 'Married':
        return salary * 8 / 100
    else:
        return 0
df['Subsidy (Married)'] = df.apply(lambda row: assign_base_salary_marital(row['Salary Base'],row['MaritalDesc']), axis=1)

# Assign Base Salary Bonus or Cut Based on Performance Score
def assign_base_salary_perf_score(salary,perf):
    if perf == 'Fully Meets':
        return salary * 4 / 100
    elif perf == 'Exceeds':
        return salary * 2 / 100    
    elif perf == 'Needs Improvement':
        return -1 * (salary * 1 / 100)
    elif perf == 'PIP':
        return -1 * (salary * 3 / 100)
df['Subsidy (Performance Score)'] = df.apply(lambda row: assign_base_salary_perf_score(row['Salary Base'],row['Performance Score']), axis=1)

# Assign Base Salary Bonus Based on Years Worked
df['StartDate'] = pd.to_datetime(df['StartDate'], format='%d-%b-%y')
df['CurrDate'] = pd.to_datetime(df['ExitDate'], format='%d-%b-%y', errors='coerce')
df['CurrDate'] = df['CurrDate'].fillna(pd.Timestamp.today())
# Calculate days worked
df['DaysWorked'] = (df['CurrDate'] - df['StartDate']).dt.days
df['YearsWorked'] = round((df['CurrDate'] - df['StartDate']).dt.days / 365, 2)

def assign_years_worked_bonus(salary,year):
    if year <= 1:
        return 0
    else:
        return round(salary * (year / 1.5) / 100,1)
        
df['Subsidy (Years Worked)'] = df.apply(lambda row: assign_years_worked_bonus(row['Salary Base'],row['YearsWorked']), axis=1)

# Total Salary
df["Final Salary"] = df["Salary Base"] + df["Subsidy (Employee Type)"] + df["Subsidy (Married)"] + df["Subsidy (Performance Score)"] + df["Subsidy (Years Worked)"]

# df[['FullName','Salary Base','Subsidy (Employee Type)','Subsidy (Married)','Subsidy (Performance Score)','Subsidy (Years Worked)','Final Salary']]
# Descriptive Statistic - The 7 Employee With Highest and Lowest Latest Salary
print("\n7 Employee with Last Highest Salary\n")
print(df[['FullName','Title','Salary Base','Final Salary']].sort_values(by='Final Salary',ascending=False).head(7))

print("\n7 Employee with Last Lowest Salary\n")
print(df[['FullName','Title','Salary Base','Final Salary']].sort_values(by='Final Salary',ascending=True).head(7))


7 Employee with Last Highest Salary

             FullName            Title  Salary Base  Final Salary
1920     Karley Novak  President & CEO    100000000   122693333.3
1877   Marlon Stanton  President & CEO    100000000   121720000.0
3119  Clinton Brennan  President & CEO    100000000   120800000.0
1941  Clinton Brennan  President & CEO    100000000   120800000.0
1978   Emily Davidson  President & CEO    100000000   120100000.0
2014   Camden Mcclure  President & CEO    100000000   114446666.7
1996      Yael Garcia  President & CEO    100000000   108286666.7

7 Employee with Last Lowest Salary

             FullName                     Title  Salary Base  Final Salary
957     Karla Farrell  Administrative Assistant      7000000     6396250.0
950   Larissa Douglas  Administrative Assistant      7000000     6746250.0
2207   Hezekiah Mcgee  Administrative Assistant      7000000     6980050.0
2206       Zain Nolan  Administrative Assistant      7000000     7007583.3
893    Estelle Howard 